## Assignment 4 part 2, the story book

## Page with random number of images per line (from 3 to 5)

Storybooks look a lot neater with a different number of pictures per horizontal strip.

Careful... *this is **advanced** python-fu!* programming here below...

In [ ]:
import numpy as np
from IPython.display import Image
from PIL import Image as pili, ImageOps as piliops, ImageDraw as pild, ImageFont as pilf 
from PIL import ImageEnhance as pilie
#from PIL import Image, ImageDraw, ImageFont, ImageFilter
import pandas as pd
def listSum(a,b):return [sum(i) for i in list(zip(a,b))]
from PIL import ImageOps
from fpdf import FPDF

In [ ]:
############################ Step 1. From photo to drawing   #####################################

###1.1 Importing modules
import cv2
import numpy as np
import matplotlib.pyplot as plt
###1.2. Loading and plotting the original image
filename='images/original/intro_jiwoo.jpg'
temp_filename='images/temp/intro_jiwoo.jpg'
img = cv2.imread(filename)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.figure(figsize=(10,10))
plt.imshow(img)
plt.axis("off")
plt.title("Original Image")
plt.show()
###1.3. Converting Image to GrayScale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.medianBlur(gray, 5)
plt.figure(figsize=(10,10))
plt.imshow(gray,cmap="gray")
plt.axis("off")
plt.title("Grayscale Image")
plt.show()
###1.4  blur the greyscale to get rid of the details
ksize = (21, 21)
gray_blur = cv2.blur(gray, ksize, cv2.BORDER_DEFAULT) 
plt.figure(figsize=(10,10))
plt.imshow(gray_blur,cmap="gray")
plt.axis("off")
plt.title("Blurred Image")
plt.show()
###1.5. Getting edged image
gray_blur = gray_blur.astype(np.uint8)
edges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 43, 7)
plt.figure(figsize=(10,10))
plt.imshow(edges,cmap="gray")
plt.axis("off")
plt.title("Edged Image")
plt.show()
###1.6. Turning Images into Cartoons
blur = cv2.blur(img, ksize, cv2.BORDER_DEFAULT) 
color = cv2.bilateralFilter(blur, 9, 250, 250)
cartoon = cv2.bitwise_and(color, color, mask=edges)
plt.figure(figsize=(10,10))
##img = image.img_to_array(img, dtype='uint8')
plt.imshow(cartoon,cmap="gray")
plt.axis("off")
plt.title("Cartoon Image")
plt.show()

In [ ]:
###################1.7 define function to do the above all together:

filename='images/original/intro_hakseong.jpg'
temp_filename='images/temp/intro_hakseong.jpg'
img = cv2.imread(filename)
def step_1(img):
    ###1.7.1 Loading and plotting the original image
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10,10))
    ###1.7.2 Converting Image to GrayScale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray, 5)
    ###1.7.3  blur the greyscale to get rid of the details
    ksize = (21, 21)
    gray_blur = cv2.blur(gray, ksize, cv2.BORDER_DEFAULT) 
    ###1.7.4 Getting edged image
    gray_blur = gray_blur.astype(np.uint8)
    edges = cv2.adaptiveThreshold(gray_blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 43, 7)
    ###1.7.6 Turning Images into Cartoons
    blur = cv2.blur(img, ksize, cv2.BORDER_DEFAULT) 
    color = cv2.bilateralFilter(blur, 9, 250, 250)
    cartoon = cv2.bitwise_and(color, color, mask=edges)
    cartoon = cv2.cvtColor(cartoon,cv2.COLOR_BGR2RGB)
    return cartoon
img=step_1(img)
cv2.imwrite(temp_filename, img)
plt.figure(figsize=(10,10))
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.axis("off")
plt.title(temp_filename)
plt.show()


In [ ]:
############################ Step 2. Add black box and the caption   #####################################

print(img.shape)
# Start coordinate, here (5, 5)
# represents the top left corner of rectangle
# Line thickness of 2 px
thickness = 250
start_point = (int(thickness/2) , int(thickness/2))
# Ending coordinate, here (220, 220)
# represents the bottom right corner of rectangle
end_point = (int(img.shape[1]-thickness/2), int(img.shape[0]-thickness/2)) # format: (heigh,width)
#color = (80, 40, 70)  #RGB, purple-ish color
color = (150, 120, 60)  #RGB, brownish color
# Using cv2.rectangle() method
# Draw a rectangle with blue line borders of thickness of 2 px
inbox = cv2.rectangle(img, start_point, end_point, color, thickness)
# Displaying the image 
plt.imshow(inbox)
plt.axis("off")
plt.title(temp_filename)
plt.show()

In [ ]:

### convert to pillow for text box adding
pil_image=pili.fromarray(inbox)
line="""Hakseong Lee is a genius Mathematician  
in North Korea. He chose to defect to  
South Korea and conceal his identity. """
#textbox_ratio=(0.9,0.2) #The ratio of the text box vs the image, 0.9 times the width and 0.2 times the height
#textbox_size=tuple(pil_image.size[0]*0.9,pil_image.size[1]*0.2)

font_Names = "fonts/arial.ttf"

def ReduceOpacity(im, opacity):
    """
    Returns an image with reduced opacity.
    Taken from http://aspn.activestate.com/ASPN/Cookbook/Python/Recipe/362879
    """
    assert opacity >= 0 and opacity <= 1
    if im.mode != 'RGBA':
        im = im.convert('RGBA')
    else:
        im = im.copy()
    alpha = im.split()[3]
    alpha = pilie.Brightness(alpha).enhance(opacity)
    im.putalpha(alpha)
    return im
def Add_text(img,text,font_Names,color,x,y,language="",**args):
    #create a new layer, set to transparent
    TINT_COLOR = (0, 0, 0) #white
    #change mode for alpha_composite()
    if img.mode != 'RGBA':
        img = img.convert('RGBA')
    else:
        img = img.copy()
    #build overlay layer, transparent    
    overlay = pili.new('RGBA',img.size, TINT_COLOR)
    overlay = ReduceOpacity(overlay,0)
    #estimate the text box size
    textbox_ratio=(0.9,0.2) #The ratio of the text box vs the image, 0.9 times the width and 0.2 times the height
    textbox_size=int(pil_image.size[0]*textbox_ratio[0]),int(pil_image.size[1]*textbox_ratio[1])
    #print("pciture_size= ", img.size, " textbox_size= ", textbox_size)
    num_lines = len(text.split('\n'))
    num_cols = max(len(i) for i in text.split('\n'))
    font_size=min(int(textbox_size[1]/num_lines),int(textbox_size[0]/num_cols))*2 # I don't know why *2, but it is proper, tested.
    #print("num_lines =", num_lines," num_cols =", num_cols, " font_size= ", font_size, )
    #font=pilf.truetype(font_Names, font_size,encoding="unic")
    font=pilf.truetype(font_Names, font_size)
        #w, h = font.getsize(text) # text size
    x_0, y_0 = (int(img.size[0]*x), int(img.size[1]*y)) #position for the black box
    x_1, y_1 = (int(img.size[0]*x), int(img.size[1]*y)) # top-left corner of the text
    margin=0.01
    TRANSPARENCY=0.5
    OPACITY = int(255 * TRANSPARENCY)
    draw = pild.Draw(overlay)
    draw.rectangle((x_0, y_0, int(x_0 + font_size*num_cols*(1+margin)), int(y_0 + font_size*(num_lines)*(1+margin))), fill=TINT_COLOR+(OPACITY,))
    #draw.rectangle([font_size*i for i in draw.textbbox((x_1, y_1),text)], fill=TINT_COLOR+(OPACITY,))
    draw.text((x_1, y_1), text, fill=color, font=font, align='center', language=language)
    #draw.text((x_1, y_1), text, fill=color, font=font,anchor="tl", align='center')
    img = pili.alpha_composite(img, overlay)
    return img
color=(255,255,255)
addText=Add_text(pil_image,line,font_Names,color,0.01,0.01)    
#display(addText)
addText=np.array(addText) 
#addText=cv2.cvtColor(addText, cv2.COLOR_RGB2BGR)  
plt.imshow(addText)
#plt.axis("off")
plt.title("Hakseong Lee 1 ")
plt.show()
#img.save('Assignment4_part1_Team_1_poster.jpg')

In [ ]:
def step_2(img, margin_color,text_line,text_color,font_Names,x,y,language=""):
    """
    This is for adding margin and caption to the image.
    """
    thickness = int((250.0/3024.0)*min(img.shape[0],img.shape[1]))
    thickness=250
    start_point = (int(thickness/2) , int(thickness/2))
    end_point = (int(img.shape[1]-thickness/2), int(img.shape[0]-thickness/2)) # format: (heigh,width)
    #color = (80, 40, 70)  #RGB, purple-ish color
    #color = (150, 120, 60)  #RGB, brownish color
    inbox = cv2.rectangle(img, start_point, end_point, margin_color, thickness)
    pil_image=pili.fromarray(inbox)
    addText=Add_text(pil_image,text_line,font_Names,text_color,x,y,language=language)
    addText=np.array(addText) 
    return addText
    
def step_1_2(img, text_line, font_Names,y=0.01,language=""): 
    """
    img is a cv2 style image matrix object
    textline is the caption of the image
    y represent the position of the text box on the y axis (0.01 top, 0.89 bottom)
    """
    margin_color = (105, 135, 195)  #105, 135, 195 |  150, 120, 60  | (101, 67, 33)
    text_color=(255,255,255)
    #font_Names="fonts/arial.ttf"
    #x,y = 0.01,0.01 top
    #x,y = 0.01,0.89 bottom
    x=0.01
    return cv2.cvtColor(step_2(step_1(img),margin_color,text_line,text_color,font_Names,x,y,language=language), cv2.COLOR_BGR2RGB)

text_line="""Jiwoo is a high school student. 
He struggled in math. But, he met Mr. Hakseong.
Then everything changed..."""
filename='images/original/intro_jiwoo.jpg'
temp_filename='images/temp/intro_jiwoo.jpg'
img = cv2.imread(filename)  
#img = step_1_2(img,text_line,0.89)  
img = step_1_2(img,text_line,'fonts/arial.ttf')  
#img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img)
plt.imsave(temp_filename,img)
plt.axis("off")
plt.title("Han Jiwoo 1 ")

plt.show()

In [ ]:
#####Step 2 final ############
## use function step_1_2(img, text_line, y=0.01) to process any picture
text_line="""Hakseong Lee is a genius Mathematician  
in North Korea. He chose to defect to  
South Korea and conceal his identity. """
filename='images/original/intro_hakseong.jpg'
temp_filename='images/temp/intro_hakseong.jpg'
img = cv2.imread(filename)
img=step_1_2(img, line,"fonts/arial.ttf")
plt.imshow(img)
plt.imsave(temp_filename,img)
plt.axis("off")
plt.title("hakseong 1 ")

plt.show()


In [ ]:
######### read a csv file and produce all cartoonized pictures #########

import pandas as pd
df = pd.read_csv('captions.csv',encoding = 'unicode_escape')
list_csv = df.values.tolist()
#print(df)
#dict_captions=df.to_dict(orient='dict',into=dict)
#print(dict_captions)
######## list_csv has 4 colums,   index, file_name, caption, mount point



In [ ]:
#list_csv  ##################### this is for cartoonizing all the pictures in original and save in temp ##########
for pic in list_csv: ###pic[0]=index; pic[1]=file_name; pic[2]=Eng_caption, pic[3]=CHN_caption, pic[4]=HIN_caption, pic[5]=mount point
    filename='images/original/'+pic[1]
    temp_filename='images/temp/'+pic[1]
    print(filename)
    img = cv2.imread(filename)
    #img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    line=pic[2]
    img=step_1_2(img, line,"fonts/arial.ttf")
    plt.imsave(temp_filename,img)
    plt.close()


In [ ]:

##############Chinese caption####################
message=u"""这是一个数学成绩不太好的男孩 Jiwoo 的故事。
还有一个极力隐藏自己身份的数学家学城先生。
和一个充满责任感和诚实的女孩。
Jiwoo：我发现很难跟上这个艰苦的课程，尤其是在数学方面
Jiwoo：这个班全是天才，我不是其中之一
校长/老师：我觉得你应该转学，你的成绩下降得很快
Jiwoo：我怎么能告诉我妈妈我数学课不及格呢？
老师：Jiwoo 昨晚拿着啤酒瓶被抓了，你拿这些给谁？
Jiwoo：这是给我的。没有别人了 ：（
用啤酒瓶抓到智宇的保安再次抓到他潜伏
我现在只能四处游荡。天快黑了。老保安或看门人！人们说他是朝鲜的缺陷……该死的共产党员！毕竟，他是罪魁祸首！什么？他又来了？！
保安/看门人 Hakseong 先生同意让他晚上陪在他身边
Hakseong：那些是什么？这些是他的数学作业吗？让我看一看。
Hakseong 彻夜坐着在 Jiwoo 不知情的情况下解决他的家庭作业问题
第二天，在课堂上 Jiwoo 意识到他所有的解决方案都是正确的！这怎么发生的？
Jiwoo 意识到可能是保安/看门人在他睡着的晚上解决了他们
Jiwoo再次求Hakseong，但这次是教他数学
为了说服学城老师教他，智宇还买了一些自己喜欢的饮料放在冰箱里
最后学成先生同意了，说要教他
Hakseong：我会教你。但你不敢告诉任何人
你不应该问我除了数学之外的任何问题。
而且，如果您只是为了获得更高的成绩而学习，那就算了。
Hakseong 先生教导 Jiwoo 专注于真相，而不是答案。因为如果你只关注答案，你就会忽视真相。数学是关于真理的。
学习数学的方法就是和它交朋友。那么，为什么不手动计算平方根 2 到小数点后第 200 位呢？
Hakseong先生通过计算根教Jiwoo与数字交朋友
而有时候，学城也没有那么“冷”。谈到数学，Hakseong 充满了热情。他可以花几个小时崇拜数学公式，一种叫做“欧拉”的东西
Boram：那些人都在逃避他们的错误，你为什么要包庇他们？
Jiwoo：没关系，我会没事的！
宝蓝对智宇在课堂上的表现感到惊讶
宝蓝四处嗅探，发现智宇正在和一个老人一起解数学题
她承诺永远不会告诉任何人，试图让自己留在 Hakseong 老师的班级里
hakseong终于放弃了，让她留下来教她数学
hakseong：数学很美！圆周率可以在钢琴上弹奏！
hak 教 boram 可以在钢琴上学习 pi，如果每个音符都可以用数字代替
小数点后的数字可以在钢琴上演奏成优美的旋律
数学奥林匹克竞赛是一项决定性的考试。通过考试的学生将被允许毕业。
奥林匹克竞赛是一场艰难的考试。每个人都开始为此做准备。
Boram 被迫参加导师“Euler Studio”课程，为数学奥林匹亚做准备。
她对这种独家学费感到不舒服
Jiwoo 继续接受 Hakseong 先生的指导。
Jiwoo、Hakseong 和 Boram 继续上课
许多天过去了，日子变成了几个月
Jiwoo 的数学要好得多，他甚至可以和老师争论解决方案
Euler Studio 开始教 Boram 如何解决一组特定的问题。
在奥林匹克竞赛的那天，她意识到这些问题与她在补习班中解决的问题完全相同
Boram 对父母让她陷入的境地感到愤怒和极度羞愧。这是作弊！她悲痛欲绝，当场放弃了奥林匹克竞赛。
Boram 通过 Facebook 页面以匿名方式通知有关人员泄露的问题
数学老师指责智宇泄露试卷
当当局看到 Jiwoo 进入打印机室时，Jiwoo 无法为自己辩护
他被学校开除并被迫转到社区学校
Jiwoo 转校后面临新的挑战
可智宇其实是在打印一份数学论文给学成先生看，满足学成先生对数学的渴望。
boram 相信真理，她无法忍受在一个如此不诚实的学校里，所以她加入了 Jiwoo 转学的学校。
这所学校没有太多的天才，数学课也不是很难
随着数学难度的降低，以及他与 Hakseong 的常规课程，Jiwoo 发现他能够在课堂上取得很好的成绩
凭借从 Hakseong 老师那里学到的知识，Jiwoo 开始帮助他的朋友们学习数学。
谁需要他的帮助，他就愿意“走第二里路”。
但他能够很好地适应。
智宇和鹤城老师去书店，智宇帮他做了这么多，给他买了一个毛绒玩具作为礼物。
在书店，有人认出了这位数学家！
hakseong 后来意识到他正受到朝鲜政府的监视
朝鲜声称韩国绑架了他们解决了黎曼定理的顶尖科学家
莱曼定理的证明对数学界来说是一场地震，它将终结目前基于素数的密码系统，并可能成为朝鲜政府改变世界的有力武器。
所以他决定逃跑！
他乘坐早上的第一班火车离开了这座城市。
Jiwoo 和 Boram 担心 Hakseong 先生，他们出去寻找他。
他们到处寻找，但找不到Hakseong先生。
很快，他们意识到我们也在监视他们的朝鲜特工……
所以他们认为他们需要做点什么来帮助 Hakseong 先生。
于是他们决定分手！但他们继续寻找他们心爱的导师。
宝蓝找到学成先生，告知他目前的情况，智宇被指控泄露论文，已被开除
Hakseong 先生意识到，为了为 Jiwoo 伸冤，他需要暴露自己。
Hakseong 写信给他的数学家朋友寻求帮助。
Hakseong 先生决定去 Jiwoo 的学校与数学老师对质。
有了智宇打印的试卷，鹤城先生得以与他放在智宇身上的画框里的数学老师对峙。
这位数学家朋友通过他的关系得知原来欧拉工作室的幕后黑手就是数学老师，并通过泄露奥林匹亚题大赚了一笔。
数学老师泄露奥数试卷被骂！
学校董事会决定惩罚数学老师并有效解雇他。这个决定也让智宇回到了学校。
Hakseong 先生仍然不确定自己的未来，因为他担心朝鲜特工可能很快就会再次出现。
然而，他的老朋友，附近的回收商，实际上是韩国的秘密特工，负责保护他，
已收到政府命令，将 Hakseong 先生送往德国进行最终保护。
原来连假护照都准备好了。
再见了，鹤城先生。希望有一天我们能重逢。
Jiwoo 在学习上表现出奉献和努力，一直取得高分，并以他对数学的理解给老师留下了深刻的印象。
他的努力得到了回报，他的学术成就得到了认可，并为未来的成功奠定了坚实的基础。
他以优异的成绩毕业了！每个人都为他的学术成就鼓掌
朝鲜特工非常沮丧，以至于他们无法再联系到 Hakseong 先生。
Jiwoo 最终在著名的首尔国立大学找到了一份数学研究员的工作。
他很快在学术界声名鹊起。
他的工作重点是解决复杂的数学问题和开发该领域的新理论。
机会终于来了，韩国和德国政府之间有一个合作计划。太想念鹤城先生了，智宇申请了，希望有幸见到鹤城先生。
他终于能够踏上这个诞生了许多伟大数学家的欧洲国家。
在他的一次工作旅行中，Jiwoo 有机会在德国的 Max-Planck Gesellschaft 会见了 Hakseong 先生。
Jiwoo 从未忘记 Hakseung 先生对他职业生涯的影响。他很感激从 Hakseung 那里学到的东西，无论是在学术上还是在个人方面。
Jiwoo非常怀念在Hakseong先生指导下的时光。
Jiwoo 将他在该领域的成功很大程度上归功于他从 Hakseung 那里学到的教训。他相信与 Hakseung 的合作和思想交流帮助他成长为一名数学家，并促使他在工作中追求卓越。
信不信由你，Boram 也找到了去德国的路。她非常想念她的老朋友。没有它们，数学对她来说就没有那么有趣了。
宝蓝希望她能和鹤城先生和智宇一起弹更多的钢琴。
Jiwoo 和 Hakseong 先生以及 Boram 证明了真实和诚实是使生活和友谊持久的最有价值的东西。坚持真理才是真正的忠诚。 QED"""

message=message.split('\n')
id=0
for line in message:
    Lx=str('')
    while len(line)>0:
        Lx=Lx+line[:10]+'\n'
        line=line[10:]
    message[id]=Lx 
    id+=1
len(message)
message

In [ ]:
#################### Chinese version##########################
def Add_text_CHN(img,text,font_Names,color,x,y,language="",**args):
    #create a new layer, set to transparent
    TINT_COLOR = (0, 0, 0) #white
    #change mode for alpha_composite()
    if img.mode != 'RGBA':
        img = img.convert('RGBA')
    else:
        img = img.copy()
    #build overlay layer, transparent    
    overlay = pili.new('RGBA',img.size, TINT_COLOR)
    overlay = ReduceOpacity(overlay,0)
    #estimate the text box size
    textbox_ratio=(0.9,0.2) #The ratio of the text box vs the image, 0.9 times the width and 0.2 times the height
    textbox_size=int(pil_image.size[0]*textbox_ratio[0]),int(pil_image.size[1]*textbox_ratio[1])
    #print("pciture_size= ", img.size, " textbox_size= ", textbox_size)
    num_lines = len(text.split('\n'))
    num_cols = max(len(i) for i in text.split('\n'))
    font_size=int(min(int(textbox_size[1]/num_lines),int(textbox_size[0]/num_cols))*0.6) # I don't know why not *2, but it is proper, tested.
    #print("num_lines =", num_lines," num_cols =", num_cols, " font_size= ", font_size, )
    #font=pilf.truetype(font_Names, font_size,encoding="unic")
    font=pilf.truetype(font_Names, font_size)
        #w, h = font.getsize(text) # text size
    x_0, y_0 = (int(img.size[0]*x), int(img.size[1]*y)) #position for the black box
    x_1, y_1 = (int(img.size[0]*x), int(img.size[1]*y)) # top-left corner of the text
    margin=0.01
    TRANSPARENCY=0.5
    OPACITY = int(255 * TRANSPARENCY)
    draw = pild.Draw(overlay)
    draw.rectangle((x_0, y_0, int(x_0 + font_size*num_cols*(1+margin)), int(y_0 + font_size*(num_lines)*(1+margin))), fill=TINT_COLOR+(OPACITY,))
    #draw.rectangle([font_size*i for i in draw.textbbox((x_1, y_1),text)], fill=TINT_COLOR+(OPACITY,))
    draw.text((x_1, y_1), text, fill=color, font=font, align='center', language=language)
    #draw.text((x_1, y_1), text, fill=color, font=font,anchor="tl", align='center')
    img = pili.alpha_composite(img, overlay)
    return img

def step_2_CHN(img, margin_color,text_line,text_color,font_Names,x,y,language=""):
    """
    This is for adding margin and caption to the image.
    """
    thickness = int((250.0/3024.0)*min(img.shape[0],img.shape[1]))
    thickness=250
    start_point = (int(thickness/2) , int(thickness/2))
    end_point = (int(img.shape[1]-thickness/2), int(img.shape[0]-thickness/2)) # format: (heigh,width)
    #color = (80, 40, 70)  #RGB, purple-ish color
    #color = (150, 120, 60)  #RGB, brownish color
    inbox = cv2.rectangle(img, start_point, end_point, margin_color, thickness)
    pil_image=pili.fromarray(inbox)
    addText=Add_text_CHN(pil_image,text_line,font_Names,text_color,x,y,language=language,)
    addText=np.array(addText) 
    return addText
    
def step_1_2_CHN(img, text_line, font_Names,y=0.01,language=""): 
    """
    img is a cv2 style image matrix object
    textline is the caption of the image
    y represent the position of the text box on the y axis (0.01 top, 0.89 bottom)
    """
    margin_color = (105, 135, 195)  #105, 135, 195 |  150, 120, 60  | (101, 67, 33)
    text_color=(255,255,255)
    #font_Names="fonts/arial.ttf"
    #x,y = 0.01,0.01 top
    #x,y = 0.01,0.89 bottom
    x=0.01
    return cv2.cvtColor(step_2_CHN(step_1(img),margin_color,text_line,text_color,font_Names,x,y,language=language), cv2.COLOR_BGR2RGB)


for pic in list_csv: ###pic[0]=index; pic[1]=file_name; pic[2]=caption, pic[3]=mount point
    filename='images/original/'+pic[1]
    temp_filename='images/temp_CHN/'+pic[1]
    text=message[int(pic[0])]
    print(pic[0],":",filename," : ",text)
    img = cv2.imread(filename)
    #img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    line=text  #Chines is pic[3]
    #draw.text ( (10,10), unicode_text, font=unicode_font, fill=font_color )
    img=step_1_2_CHN(img, line,"fonts/Dengl.ttf",language="zh-CN")
    plt.imsave(temp_filename,img)
    plt.close()


#languages = ["", "zh", "zh-Hans", "zh-CN", "zh-TW", "zh-HK"]

In [ ]:
################################### Step 3, arrange the pages  #########################
# This part of code will generate a randon distribution of images on a page.
list_im = [str(x) for x in range(99)]
list_lines=list()   
while 0 < len(list_im):
    lines = 0
    print("new page!")
    id_page = 0
    list_lines.append("new page!")
    num_cols = old_num_cols = 0
    while lines < 5 and 0 < len(list_im):
        lines += 1
        while num_cols == old_num_cols:
            num_cols = np.random.randint(3, 5)
        old_num_cols = num_cols
        window = list_im[:num_cols]
        print(window)
        list_lines.append(window)
        list_im = list_im[num_cols:]
def make_layout():
    list_layout=list()
    list_page=list()
    for i in list_lines:
        if i=='new page!':
            list_layout.append(list_page)
            list_page=list() 
        else:
            list_page.append(i)
    list_layout.append(list_page)
    list_layout=list_layout[1:]
    return list_layout
list_layout=make_layout()


In [ ]:
### step 3.1 create a letter size canvas, 400dpi ###
letter_paper_size = (8.5,11) # American letter paper size, inches
resolution = 400 #dpi
canvas_size=[int(i*resolution) for i in letter_paper_size]
#print(canvas_size)  #[3400, 4400]
page_margin_ration=0.04
page_margin_width = int(min(*canvas_size)*page_margin_ration*0.5)
#print(page_margin_width)  #68
### step 3.2 generate the position and resize information for the pictures.
dict_pos=dict()
for i in list_layout:                                                       # per page, working on page i
    num_lines=len(i)                                                        # find the number of lines in page i
    height_lines= int((canvas_size[1]-page_margin_width*2)/num_lines)       # find the height of lines in page i
    page_origin = [page_margin_width,page_margin_width]                     # the original position of content on the page
    id_line=0                                                               # 
    for j in i:                                                             # per line, working on line j
        line_origin = listSum(page_origin,[0,id_line*height_lines])         # def listSum(a,b):return [sum(i) for i in list(zip(a,b))]
        #print("line_origin= ",line_origin)
        num_cols = len(j)                                                   # find the number of columns in line j
        width_cols = int((canvas_size[0]-page_margin_width)/num_cols)       # find the width of columns in line j
        id_line+=1                                                          # id of line
        id_box = 0                                                          # id of box in line j
        for k in j:                                                         # per picture, in individual bounding box id
            mount_point = listSum(line_origin,[id_box*width_cols,0]) 
            #mount_point = listSum([id_box*width_cols,0],line_origin)        #
            mount_box = [width_cols,height_lines]  
            #mount_box = [height_lines,width_cols]                           #
            dict_pos.update({k:(mount_point,mount_box)})                    # add the position info to dict_pos
            id_box+=1                                                       #
for i in dict_pos:  
    if int(i)<2 or int(i)>97:
        print(i,":",dict_pos[i]) 
###### page arranging done: dict_pos is the dictionary that stores the mounting point coordinate and bounding box size of each picture

In [ ]:
########################## Put temp images into canvas #############################

page_id=0
for page in list_layout:
    #print(page)
    canvas = pili.new('RGBA',canvas_size, (101, 67, 33)) # coffee 111, 78, 55; dark brown 101, 67, 33
    canvas = ReduceOpacity(canvas,1)
    page_id+=1
    #canvas
    for line in page:
        #print(line)
        for box in line:
            #print(box)
            #filename = list_csv[int(box)][1]
            filename='images/temp/'+list_csv[int(box)][1]
            print(box, ":",filename)
            img=pili.open(filename)#.convert(mode='RGBA')
            #img=pili.fromarray(img)
            img = ImageOps.contain(img, dict_pos[box][1]) # change to bounding box size
            canvas.paste(img, dict_pos[box][0])
    page_name=  'images/pages/'+str(page_id)+'.jpg' 
    canvas=canvas.convert('RGB')     
    canvas.save(page_name, quality=95)

            


In [ ]:
########################## Put temp images into canvas #############################

page_id=0
for page in list_layout:
    #print(page)
    canvas = pili.new('RGBA',canvas_size, (101, 67, 33)) # coffee 111, 78, 55; dark brown 101, 67, 33
    canvas = ReduceOpacity(canvas,1)
    page_id+=1
    #canvas
    for line in page:
        #print(line)
        for box in line:
            #print(box)
            #filename = list_csv[int(box)][1]
            filename='images/temp_CHN/'+list_csv[int(box)][1]
            print(box, ":",filename)
            img=pili.open(filename)#.convert(mode='RGBA')
            #img=pili.fromarray(img)
            img = ImageOps.contain(img, dict_pos[box][1]) # change to bounding box size
            canvas.paste(img, dict_pos[box][0])
    page_name=  'images/pages_CHN/'+str(page_id)+'.jpg' 
    canvas=canvas.convert('RGB')     
    canvas.save(page_name, quality=95)


In [ ]:
##################### Lastly, making pdf ############################

#pdf = FPDF()
width, height = letter_paper_size
pdf = FPDF(unit = "pt", format = [width, height])
    
# imagelist is the list with all image filenames
imagelist = [ 'images\\pages\\'+str(i)+'.jpg' for i in range(1,7)]

for image in imagelist:
    pdf.add_page()
    pdf.image(image, 0, 0, width, height)
pdf.output("In_Our_Prime.pdf", "F")
print("done!")

In [ ]:
##################### Lastly, making pdf ############################

#pdf = FPDF()
width, height = letter_paper_size
pdf = FPDF(unit = "pt", format = [width, height])
    
# imagelist is the list with all image filenames
imagelist = [ 'images\\pages_CHN\\'+str(i)+'.jpg' for i in range(1,7)]

for image in imagelist:
    pdf.add_page()
    pdf.image(image, 0, 0, width, height)
pdf.output("In_Our_Prime_CHN.pdf", "F")
print("done!")


canvas_size=[3400, 4400]
margin=min(canvas_size)*.02
height_lines=int((canvas_size[1]-margin*2)/5) ## keep 5 lines.
width_page=int(canvas_size[1]-margin*2)      ## get rid of 2 side margins
list_name=[pics[1] for pics in list_csv]
img_id=0
page_id=0
while len(list_name)>0:
    canvas = pili.new('RGBA',canvas_size, (101, 67, 33)) # coffee 111, 78, 55; dark brown 101, 67, 33
    canvas = ReduceOpacity(canvas,1)
    line_id=0
    while line_id<5:
        if img==[]: 
            dict_image={} 
        else: 
            dict_image={list_name[0]:[img,img.size]}
        filename='images/temp_CHN/'+list_name[0]
        list_name=list_name[1:]                    # throw away the 1st term of list_name, util the list is empty
        print(img_id, ":",list_name[0])
        img=pili.open(filename)
        w, h = img.width, img.height            # get the width and height of the image
        ratio = height_lines/h
        w, h = int(w*ratio), int(h*ratio)
        img= img.resize((w,h))
        img_id+=1
        width_remain=width_page
        while width_remain>w and len(list_name)>0:
            dict_image.update({list_name[0]:(img,img.size)})             # add the image to the list_image of this line
            img=[]
            width_remain-=w
            filename='images/temp_CHN/'+list_name[0]
            list_name=list_name[1:] 
            print(img_id, ":",list_name[0])
            img=pili.open(filename)
            w, h = img.width, img.height            # get the width and height of the image
            ratio = height_lines/h
            w, h = int(w*ratio), int(h*ratio)
            img= img.resize((w,h))
            img_id+=1
        width_slab=int(width_remain/(len(dict_image)+1))
        x,y =margin, margin+line_id*h
        for img in dict_image:
            x,y= x+width_slab+img.width, y
            canvas.paste(img, (x,y,x+img.width,y+img.height))
            img.close
        line_id+=1
    page_name=  'images/pages_CHN/'+str(page_id)+'.jpg' 
    canvas=canvas.convert('RGB')     
    canvas.save(page_name, quality=95)
    page_id+=1



## Bibliography


Turn Images into Cartoons using Python, By Isha Bansal / May 31, 2021   
https://www.askpython.com/python/examples/images-into-cartoons  


Add text to the picture with openCV   
https://www.geeksforgeeks.org/python-opencv-cv2-puttext-method/


To convert from OpenCV image to PIL image use:    
https://stackoverflow.com/questions/43232813/convert-opencv-image-format-to-pil-image-format



Pillow image to OpenCV image:

cv2_img = np.array(pil_img)  
cv2_img = cv2.cvtColor(cv2_img, cv2.COLOR_RGB2BGR)

OpenCV image to Pillow image:

cv2_img = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB)  
pil_img = Image.fromarray(cv2_img)


Load CSV data into List and Dictionary using Python  
https://www.geeksforgeeks.org/load-csv-data-into-list-and-dictionary-using-python/

Pandas is easier    
https://www.w3schools.com/python/pandas/pandas_csv.asp


pandas.DataFrame.to_dict   
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html